In [6]:
import cv2
from os.path import basename
from glob import glob
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageChops

In [16]:
def map_mask(imarray):
    ## Crop black borders before map feature dectection
    gray = cv2.cvtColor(imarray, cv2.COLOR_BGR2GRAY)
    _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
    contours,hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnt = contours[0]
    x,y,w,h = cv2.boundingRect(cnt)

    crop = imarray[y:y+h,x:x+w]

    ## EDGE DETECTION
    gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (11, 11), 0)
    # Edge Detection.
    canny = cv2.Canny(gray, 0, 200)
    canny = cv2.dilate(canny, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))
    
    # Finding contours for the detected edges.
    contours, hierarchy = cv2.findContours(canny, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    # Keeping only the largest detected contour.
    contour = sorted(contours, key=cv2.contourArea, reverse=True)[0]

    # Mask out everything that is not the map
    mask = np.zeros_like(crop)
    cv2.fillPoly(mask, pts=[contour], color=(255,255,255))
    masked_crop = cv2.bitwise_and(crop, mask)

    # Reapply black borders to maintain correct image size
    masked_img = np.zeros_like(imarray)
    masked_img[y:y+h,x:x+w] = masked_crop

    return masked_img

In [ ]:
filePath = '/home/shared/DARPA/training/'
trainSource = 'AK_McCarthy.tif'
sourceImg = cv2.imread(filePath+trainSource)

In [ ]:
maskedImg = map_mask(sourceImg)

fig, axs = plt.subplots(1, 2, figsize=(15, 8))
axs[0].imshow(sourceImg)
axs[1].imshow(maskedImg)

plt.show()